## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [7]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [8]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [9]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [10]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [11]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [12]:
len(sentences_ted)

266694

In [15]:
print(sentences_ted[0])
print(sentences_ted[1])
print(sentences_ted[2])
print(sentences_ted[3])
print(sentences_ted[4])


['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']
['consider', 'facit']
['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [19]:
from collections import Counter
word_counts = Counter()
for sentence in sentences_ted:
    for word in sentence:
        word_counts[word] += 1

In [27]:
word_counts.most_common(1000)

[('the', 207748),
 ('and', 149305),
 ('to', 125169),
 ('of', 114818),
 ('a', 105399),
 ('that', 95146),
 ('i', 83180),
 ('in', 78070),
 ('it', 74738),
 ('you', 70923),
 ('we', 67629),
 ('is', 63251),
 ('s', 57156),
 ('this', 49241),
 ('so', 37014),
 ('they', 33102),
 ('was', 30806),
 ('for', 29713),
 ('are', 27995),
 ('have', 27344),
 ('but', 26732),
 ('what', 26519),
 ('on', 25962),
 ('with', 24706),
 ('can', 23377),
 ('t', 22757),
 ('about', 21246),
 ('there', 21041),
 ('be', 20201),
 ('as', 19488),
 ('at', 19216),
 ('all', 19021),
 ('not', 18626),
 ('do', 17928),
 ('my', 17908),
 ('one', 17551),
 ('re', 17012),
 ('people', 16723),
 ('like', 16273),
 ('if', 15868),
 ('from', 15452),
 ('now', 14387),
 ('our', 14061),
 ('he', 13986),
 ('an', 13917),
 ('just', 13896),
 ('these', 13882),
 ('or', 13864),
 ('when', 13278),
 ('because', 12879),
 ('very', 12363),
 ('me', 12302),
 ('out', 12163),
 ('by', 11935),
 ('them', 11595),
 ('how', 11576),
 ('know', 11506),
 ('up', 11445),
 ('going', 1

In [25]:
counts_ted_top1000 = [x[1] for x in word_counts.most_common(1000)]

Plot distribution of top-1000 words

In [26]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [28]:
from gensim.models import Word2Vec

In [30]:
?Word2Vec

In [31]:
model_ted = Word2Vec(sentences_ted, min_count=10)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [34]:
model_ted.most_similar("man")

[('woman', 0.8443951606750488),
 ('guy', 0.8201020956039429),
 ('lady', 0.7742055654525757),
 ('girl', 0.7575396299362183),
 ('boy', 0.7521883845329285),
 ('kid', 0.7166053056716919),
 ('soldier', 0.7163606882095337),
 ('gentleman', 0.705656886100769),
 ('poet', 0.6981457471847534),
 ('rabbi', 0.6790573596954346)]

In [35]:
model_ted.most_similar("computer")

[('machine', 0.7197245955467224),
 ('software', 0.6967129707336426),
 ('robot', 0.6931082606315613),
 ('device', 0.670138418674469),
 ('program', 0.6583622694015503),
 ('chip', 0.6508647203445435),
 ('3d', 0.6381388902664185),
 ('interface', 0.6345016360282898),
 ('video', 0.6260940432548523),
 ('camera', 0.6122509241104126)]

In [36]:
model_ted.most_similar('india')

[('australia', 0.8836702704429626),
 ('europe', 0.8673836588859558),
 ('brazil', 0.8665424585342407),
 ('africa', 0.8649249076843262),
 ('america', 0.8597109913825989),
 ('china', 0.8563129901885986),
 ('afghanistan', 0.8513227105140686),
 ('kenya', 0.8454620838165283),
 ('japan', 0.8250157833099365),
 ('asia', 0.8246949911117554)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [37]:
words_top_ted = [x[0] for x in word_counts.most_common(1000)]

In [38]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [39]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [40]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [46]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [47]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [48]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [49]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [51]:
sentences_wiki[0:5]

[' the agency provided      million for debris removal and rebuilding public buildings ',
 ' the grande pr mio de s o pedro   saint peter grand prix     which occurs in the city  s streets during the summer   is part of the national calendar of the portuguese athletics federation ',
 ' drepanolobium savannahs in the dry season ',
 ' the trustees and board members are elected to four     year terms ',
 '        kilometre       ']

In [52]:
sentences_wiki_splitted = []
for sent_str in sentences_wiki:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_wiki_splitted.append(tokens)

In [54]:
sentences_wiki_splitted[0:1]

[['the',
  'agency',
  'provided',
  'million',
  'for',
  'debris',
  'removal',
  'and',
  'rebuilding',
  'public',
  'buildings']]

In [55]:
word_counts_wiki = Counter()
for sentence in sentences_wiki_splitted:
    for word in sentence:
        word_counts_wiki[word] += 1

In [56]:
word_counts_wiki.most_common(1000)

[('the', 1284646),
 ('of', 548096),
 ('and', 499701),
 ('in', 433777),
 ('to', 399166),
 ('a', 357774),
 ('was', 214934),
 ('s', 164268),
 ('on', 153765),
 ('as', 145301),
 ('for', 140016),
 ('that', 138154),
 ('with', 129558),
 ('by', 126009),
 ('is', 102410),
 ('his', 93521),
 ('at', 92337),
 ('he', 92011),
 ('from', 88800),
 ('it', 88527),
 ('were', 70749),
 ('an', 64403),
 ('had', 58547),
 ('which', 55340),
 ('be', 47803),
 ('this', 43227),
 ('but', 42211),
 ('first', 40810),
 ('are', 40764),
 ('not', 39765),
 ('their', 38671),
 ('one', 38016),
 ('after', 37842),
 ('her', 37091),
 ('also', 37001),
 ('its', 36804),
 ('they', 35107),
 ('two', 34316),
 ('or', 33420),
 ('have', 33077),
 ('who', 31776),
 ('new', 30728),
 ('been', 30484),
 ('has', 30163),
 ('she', 29142),
 ('when', 27224),
 ('time', 26404),
 ('during', 26383),
 ('other', 25770),
 ('would', 25395),
 ('into', 24985),
 ('all', 24189),
 ('i', 23765),
 ('more', 23109),
 ('over', 21329),
 ('while', 21274),
 ('him', 20203),
 ('

In [57]:
model_wiki = Word2Vec(sentences_wiki_splitted, min_count=10)

In [58]:
words_top_wiki = [x[0] for x in word_counts_wiki.most_common(1000)]

In [61]:
model_wiki.most_similar("man")

[('woman', 0.7282345294952393),
 ('girl', 0.6239097118377686),
 ('boy', 0.6187779307365417),
 ('person', 0.6003495454788208),
 ('dog', 0.5773534774780273),
 ('soldier', 0.5579779148101807),
 ('figure', 0.5414016842842102),
 ('cat', 0.5228359699249268),
 ('creature', 0.5203118920326233),
 ('someone', 0.5150976777076721)]

In [62]:
model_wiki.most_similar("computer")

[('software', 0.7863651514053345),
 ('computers', 0.735518217086792),
 ('hardware', 0.7304807901382446),
 ('user', 0.708527684211731),
 ('cgi', 0.6905224323272705),
 ('technology', 0.688383936882019),
 ('interface', 0.687021017074585),
 ('computing', 0.6771199107170105),
 ('interactive', 0.6760050654411316),
 ('controller', 0.6650823354721069)]

In [63]:
model_wiki.most_similar("india")

[('china', 0.7368907928466797),
 ('pakistan', 0.7261090278625488),
 ('africa', 0.7226329445838928),
 ('korea', 0.6859506964683533),
 ('australia', 0.6859433650970459),
 ('indonesia', 0.6736545562744141),
 ('europe', 0.66221022605896),
 ('asia', 0.64947509765625),
 ('iran', 0.641077995300293),
 ('taiwan', 0.6370406150817871)]

#### t-SNE visualization

In [59]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [60]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)